# CIFAR100 latent dimension sweep summary

This notebook combines the three sweeps and plots test accuracy vs. latent dimension with mean markers, solid connecting lines, and colored ±2 std bands for DeepLDA and Softmax. Background remains transparent with a faint grid.


In [ ]:
from pathlib import Path
import json
import statistics as stats
import matplotlib.pyplot as plt

plt.style.use("ggplot")


In [ ]:
# Load final test accuracies from the three sweep files for both models
sweep_files = [
    Path("cifar100_latent_dim_sweep1.json"),
    Path("cifar100_latent_dim_sweep2.json"),
    Path("cifar100_latent_dim_sweep3.json"),
]
model_names = ["DeepLDA", "Softmax"]

runs_by_model_dim = {m: {} for m in model_names}
for path in sweep_files:
    with path.open() as f:
        data = json.load(f)
    for model in model_names:
        for dim, metrics in data[model].items():
            runs_by_model_dim[model].setdefault(int(dim), []).append(metrics["final_test"])

summary_by_model = {}
for model, dim_map in runs_by_model_dim.items():
    latent_dims = sorted(dim_map)
    mean_acc = []
    std_acc = []
    for dim in latent_dims:
        vals = dim_map[dim]
        mean_acc.append(stats.mean(vals))
        std_acc.append(stats.pstdev(vals) if len(vals) > 1 else 0.0)

    summary_rows = []
    for dim, mean_val, std_val in zip(latent_dims, mean_acc, std_acc):
        summary_rows.append({"latent_dim": dim, "mean_test_acc": mean_val, "std": std_val, "runs": dim_map[dim]})

    summary_by_model[model] = {
        "latent_dims": latent_dims,
        "mean": mean_acc,
        "std": std_acc,
        "summary_rows": summary_rows,
    }

summary_by_model


In [ ]:
# Plot mean accuracy with solid connecting lines and colored ±2 std bands
plt.figure(figsize=(8, 5))
colors = {"DeepLDA": "C0", "Softmax": "C1"}
alpha_band = 0.18

for model in model_names:
    info = summary_by_model[model]
    dims = info["latent_dims"]
    mean_acc = info["mean"]
    std_2 = [2 * s for s in info["std"]]
    color = colors.get(model)

    lower = [m - s for m, s in zip(mean_acc, std_2)]
    upper = [m + s for m, s in zip(mean_acc, std_2)]
    plt.fill_between(dims, lower, upper, color=color, alpha=alpha_band)
    plt.plot(dims, mean_acc, marker="o", color=color, linewidth=1.5, label=f"{model} mean")

ax = plt.gca()
ax.set_facecolor('none')
plt.gcf().patch.set_alpha(0)
ax.grid(True, color="#888888", alpha=0.5, linestyle='--', linewidth=0.8)

plt.xlabel("Latent dimension (d)")
plt.ylabel("Test accuracy")
plt.title("CIFAR100: test accuracy vs latent dimension")
plt.legend()
plt.tick_params(labelsize=12)
plt.tight_layout()
plt.show()
